In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Download data

In [ ]:
# !curl https://storage.googleapis.com/dcgan_data/images_0.npy --output images_0.npy

In [ ]:
# Use subset of real images to avoid memory issues
real_images = np.load("images_0.npy")

In [ ]:
# !curl https://storage.googleapis.com/dcgan_data/coco_images.npy --output coco_images.npy

In [ ]:
coco_images = np.load("coco_images.npy")/255

In [ ]:
# !curl http://internal.henrytu.me/export-stylegan-1617977658-10000-100-swapped.npy --output stylegan_images.npy
#     sha256sum of 5f589d0045722a9f29251fce09931cbc608d1f4a273a945644f9430ba0ae11a7
# !curl https://storage.googleapis.com/dcgan_data/export-stylegan-1617977658-10000-100-swapped.npy --output stylegan_images.npy

In [ ]:
stylegan_images = np.load("stylegan_images.npy") / 255
stylegan_images = np.moveaxis(stylegan_images, 1, -1)

In [ ]:
print(np.min(stylegan_images), np.max(stylegan_images))
print(np.min(real_images), np.max(real_images))
print(np.min(coco_images), np.max(coco_images))

In [ ]:
print(stylegan_images.shape)
print(real_images.shape)
print(coco_images.shape)

In [ ]:
# Take subset of data
subset_size = 1024
stylegan_images = stylegan_images[:subset_size]
coco_images = coco_images[:subset_size]
real_images = real_images[:subset_size]

In [ ]:
print(stylegan_images.shape)
print(real_images.shape)
print(coco_images.shape)

# Visualize Images


In [ ]:
plt.rcParams["axes.grid"] = False
def display_images(images, columns, rows, randomize = True):
  w=256
  h=256
  fig=plt.figure(figsize=(columns * 3, rows * 3))

#   gs1 = gridspec.GridSpec(columns, rows)
#   gs1.update(wspace=0.025, hspace=0.05) # set the spacing between axes.

  for i in range(1, columns*rows +1):      
      img = images[i-1]
      if randomize:
        img = images[random.randint(0, images.shape[0]-1)]
      fig.add_subplot(rows, columns, i)
      
      plt.axis('off')
      plt.imshow(img)  
        
        
  fig.subplots_adjust(wspace=0, hspace=0)
  plt.axis('off')
  plt.show()

def display_images_grayscale(images, columns, rows, randomize = True, scale_brightness = False):
  w=256
  h=256
  fig=plt.figure(figsize=(columns * 3, rows * 3))

  for i in range(1, columns*rows +1):      
      img = images[i-1]
      if randomize:
        img = images[random.randint(0, images.shape[0]-1)]
    
      if scale_brightness:
        img = img * 5
            
      fig.add_subplot(rows, columns, i)  
      plt.axis('off')
      plt.imshow(img, cmap='gray', vmin=0, vmax=255)  
  fig.subplots_adjust(wspace=0, hspace=0)
  plt.axis('off')
  plt.show()

def display_image(image):  
  imgplot = plt.imshow(image)
  plt.axis('off')
  plt.show()
    
def display_image_grayscale(image):  
  imgplot = plt.imshow(image, cmap='gray', vmin=0, vmax=255)
  plt.axis('off')
  plt.show()

In [ ]:
display_images(coco_images, 4, 2)

In [ ]:
display_images(real_images, 4, 4)

In [ ]:
display_images(stylegan_images, 4, 4)

# Metrics

## Closest Image

In [ ]:
def k_nearest_images(images, test_image, k, test_image_index):
  shape = images.shape
  means = np.power(images - test_image, 2).reshape((shape[0], shape[1]*shape[2]*shape[3])).mean(axis = 1)  
  idx = means.argsort()[:k+1]  
  idx = np.setdiff1d(idx, np.array([test_image_index]))  
  print(idx)
  return images[idx]

### Coco to Coco

In [ ]:
coco_test_image_idx = 24
display_image(coco_images[coco_test_image_idx])

In [ ]:
display_images(k_nearest_images(coco_images, coco_images[coco_test_image_idx], 16, coco_test_image_idx), 4, 4, randomize=False)

### Stylegan to Stylegan

In [ ]:
stylegan_image_idx = 102
display_image(stylegan_images[stylegan_image_idx])

In [ ]:
display_images(k_nearest_images(stylegan_images, stylegan_images[stylegan_image_idx], 16, stylegan_image_idx), 4,4, randomize=False)

### Closest matches from fake Cocogan image to real ones

In [ ]:
coco_test_image_idx = 55
display_image(coco_images[coco_test_image_idx])

In [ ]:
display_images(k_nearest_images(real_images, coco_images[coco_test_image_idx], 16, 1024), 4, 2, randomize=False)

### Closest matches from fake Stylegan image to real ones

In [ ]:
stylegan_image_idx = 102
display_image(stylegan_images[stylegan_image_idx])

In [ ]:
display_images(k_nearest_images(real_images, stylegan_images[stylegan_image_idx], 16, 1024), 4,2, randomize=False)

## Find Closest Image By Color

In [ ]:
def closest_k_by_color(images, color, k, test_image_index):
    image_colors = images.reshape(images.shape[0], 256*256, 3).mean(axis=1)
    means = np.power(image_colors - color, 2).mean(axis = 1)  
    idx = means.argsort()[:k+1]  
    idx = np.setdiff1d(idx, np.array([test_image_index])) 
    return images[idx]


In [ ]:
# 105, 202, 323
coco_test_image_idx = 202
display_image(coco_images[coco_test_image_idx])

In [ ]:
stylegan_image_idx = 125

sg_image_avg_color = stylegan_images[stylegan_image_idx].reshape((256*256,3)).mean(axis=0)
print(sg_image_avg_color)

display_image(stylegan_images[stylegan_image_idx])

In [ ]:
display_images(closest_k_by_color(stylegan_images, sg_image_avg_color, 16, stylegan_image_idx), 4, 4, False)

In [ ]:
# Find noise with colors from coco 105
display_images(closest_k_by_color(coco_images, np.array([87,46,93])/255, 16, stylegan_image_idx), 4, 4, False)

In [ ]:
# coco 202 noise
display_images(closest_k_by_color(coco_images, np.array([124,193,194])/255, 16, stylegan_image_idx), 4, 4, False)

In [ ]:
# possible failure mode 0.3, 0.6, 0.6.
# Both blue and green have a peak at this point in the histograms
display_images(closest_k_by_color(coco_images, np.array([0.3,0.6,0.6]), 16, 2000), 4, 4, False)

In [ ]:
# Sky color based failure?
display_images(closest_k_by_color(stylegan_images, np.array([0.3,0.6,0.6]), 16, 2000), 4,4, False)

## Sharpness

In [ ]:
def to_grayscale(images):
  return np.mean(np.moveaxis(images, -1, 1), axis=1)

def compute_sharpness(images):
    grayscale_images = to_grayscale(images) * 255
    sharpness_values = []
    
    for i in range(grayscale_images.shape[0]):
        curr_image = grayscale_images[i]
        dy, dx = np.gradient(curr_image)
        norm = np.sqrt(dx**2 + dy**2)
        sharpness_values.append(np.mean(norm))
    
    return np.array(sharpness_values)
        

coco_sharpness = compute_sharpness(coco_images)
stylegan_sharpness = compute_sharpness(stylegan_images)
real_sharpness = compute_sharpness(real_images)

In [ ]:
print(coco_sharpness.mean(), coco_sharpness.std())
print(stylegan_sharpness.mean(), stylegan_sharpness.std())
print(real_sharpness.mean(), real_sharpness.std())

In [ ]:
def find_images_by_sharpness(images, sharpness_levels, k, mode = "min"):     
    idx = sharpness_levels.argsort()  
    if mode != "min":
        idx = np.flip(idx)
    
    idx = idx[:k+1]
        
    return images[idx], idx

def get_sharpness_images(images):
    grayscale_images = to_grayscale(images) * 255
    sharpness_images = []
    
    for i in range(grayscale_images.shape[0]):
        curr_image = grayscale_images[i]
        dy, dx = np.gradient(curr_image)
        sharpness_images.append(dx+dy)
    
    return np.array(sharpness_images)
    

In [ ]:
images, idx = find_images_by_sharpness(coco_images, coco_sharpness, 16, mode = 'max')

print(coco_sharpness[idx])
display_images(coco_images[idx], 4, 4, False)


In [ ]:
display_images_grayscale(get_sharpness_images(coco_images[idx]), 4, 4, False, True)

In [ ]:
images, idx = find_images_by_sharpness(stylegan_images, stylegan_sharpness, 16, mode = 'max')

print(stylegan_sharpness[idx])
display_images(stylegan_images[idx], 4, 4, False)


In [ ]:
display_images_grayscale(get_sharpness_images(stylegan_images[idx]), 4, 4, False, True)

In [ ]:
images, idx = find_images_by_sharpness(coco_images, coco_sharpness, 16)

print(coco_sharpness[idx])
display_images(coco_images[idx], 4, 4, False)

In [ ]:
images, idx = find_images_by_sharpness(stylegan_images, stylegan_sharpness, 16)

print(stylegan_sharpness[idx])
display_images(stylegan_images[idx], 4, 4, False)

## KL Divergence Of Color Distribution

In [ ]:
# Reference: https://machinelearningmastery.com/divergence-between-probability-distributions/

def to_grayscale(images):
  return np.mean(np.moveaxis(images, -1, 1), axis=1)

def kl_divergence(p, q):
  p = to_grayscale(p)
  q = to_grayscale(q)
  

  p = p.reshape((p.shape[0], p.shape[1] * p.shape[2]))
  q = q.reshape((q.shape[0], q.shape[1] * q.shape[2]))
  

  p_histo = np.histogram(p, 25, density=True)[0]
  q_histo = np.histogram(q, 25, density=True)[0]  

  return np.sum(p_histo * np.log2(p_histo/q_histo)), p_histo, q_histo

def color_kl_divergence(p, q, channel_num):
  # r = 0, g = 1, b = 2

  p = np.moveaxis(p, -1, 1)[:, channel_num]
  q = np.moveaxis(q, -1, 1)[:, channel_num]
  

  p = p.reshape((p.shape[0], p.shape[1] * p.shape[2]))
  q = q.reshape((q.shape[0], q.shape[1] * q.shape[2]))
  

  p_histo = np.histogram(p, 25, density=True)[0]
  q_histo = np.histogram(q, 25, density=True)[0]  

  return np.sum(p_histo * np.log2(p_histo/q_histo)), p_histo, q_histo

kl_value_red, histo_coco_red, histo_real_red = color_kl_divergence(coco_images, real_images, 0)
kl_value_green, histo_coco_green, histo_real_green = color_kl_divergence(coco_images, real_images, 1)
kl_value_blue, histo_coco_blue, histo_real_blue = color_kl_divergence(coco_images, real_images, 2)

kl_value_grayscale, histo_coco_gray, histo_real_gray = kl_divergence(coco_images, real_images)

print("RGB KL Divergence from cocogan to real")
print("R:",kl_value_red)
print("G:",kl_value_green)
print("B:",kl_value_blue)
print("Greyscale KL Divergence from cocogan to real")
print(kl_value_grayscale)

print()
print()

kl_value_red, histo_sg_red, _ = color_kl_divergence(stylegan_images, real_images, 0)
kl_value_green, histo_sg_green, _ = color_kl_divergence(stylegan_images, real_images, 1)
kl_value_blue, histo_sg_blue, _ = color_kl_divergence(stylegan_images, real_images, 2)

kl_value_grayscale, histo_sg_gray, _ = kl_divergence(stylegan_images, real_images)

print("RGB KL Divergence from stylegan to real")
print("R:",kl_value_red)
print("G:",kl_value_green)
print("B:",kl_value_blue)
print("Greyscale KL Divergence from stylegan to real")
print(kl_value_grayscale)

In [ ]:
def display_histograms(red, green, blue, gray, title):
    bins = np.linspace(0, 1, 25)
        
    
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
    fig.suptitle(title,fontsize=20)
    
    fig.set_size_inches(20, 5)
    fig.text(0.05,0.5, "Frequencies", ha="center", va="center", rotation=90,fontsize=20)
    
    ax1.plot(bins, red, color="red")
    ax1.set_xlabel('Red Pixel Values',fontsize=15)
    ax1.set_ylim([0,2])
    
    ax2.plot(bins, green, color="green")
    ax2.set_xlabel('Green Pixel Values',fontsize=15)
    ax2.set_ylim([0,2])
    
    ax3.plot(bins, blue, color="blue")
    ax3.set_xlabel('Blue Pixel Values',fontsize=15)
    ax3.set_ylim([0,2])
    
    ax4.plot(bins, gray, color="gray")
    ax4.set_xlabel('Gray Pixel Values',fontsize=15)
    ax4.set_ylim([0,2])
    
    
    plt.show()
    

In [ ]:
display_histograms(histo_sg_red, histo_sg_green, histo_sg_blue, histo_sg_gray, "StyleGAN color frequencies")

In [ ]:
display_histograms(histo_coco_red, histo_coco_green, histo_coco_blue, histo_coco_gray, "CocoGAN color frequencies")

In [ ]:
display_histograms(histo_real_red, histo_real_green, histo_real_blue, histo_real_gray, "Real Image color frequencies")

## Classifier


In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(16, (5,5), activation = 'relu', input_shape = (256,256,3)))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(32,(5,5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(5,5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(5,5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(5,5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
from sklearn.utils import shuffle

X_train = np.concatenate([real_images, coco_images[0:512], stylegan_images[0:512]])
Y_train = np.concatenate([np.ones(1024), np.zeros(512), np.zeros(512)])

X_train, Y_train = shuffle(X_train, Y_train)

In [ ]:
model.fit(X_train, Y_train, epochs=3, batch_size = 64, validation_split=0.1)

In [ ]:
results_coco = model.evaluate(coco_images[512:], np.zeros(512), batch_size=128)
print("test loss, test acc:", results_coco)

In [ ]:
results_sg = model.evaluate(stylegan_images[512:], np.zeros(512), batch_size=128)
print("test loss, test acc:", results_sg)

## PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(2) 

grayscale_real = to_grayscale(real_images)
grayscale_real = grayscale_real.reshape((grayscale_real.shape[0], grayscale_real.shape[1] * grayscale_real.shape[2]))

grayscale_coco = to_grayscale(coco_images)
grayscale_coco = grayscale_coco.reshape((grayscale_coco.shape[0], grayscale_coco.shape[1] * grayscale_coco.shape[2]))

grayscale_stylegan = to_grayscale(stylegan_images)
grayscale_stylegan = grayscale_stylegan.reshape((grayscale_stylegan.shape[0], grayscale_stylegan.shape[1] * grayscale_stylegan.shape[2]))

labels = np.concatenate((np.ones(grayscale_real.shape[0]) * 2, np.ones(grayscale_stylegan.shape[0]), np.zeros(grayscale_coco.shape[0])))
converted_data = pca.fit_transform(np.concatenate((grayscale_real, grayscale_stylegan, grayscale_coco)), y=labels)

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.figure(figsize = (12,12))
c_map = plt.cm.get_cmap('inferno', 3)
plt.scatter(converted_data[:, 0], converted_data[:, 1], s = 15,
            cmap = c_map , c = labels)
plt.colorbar()
plt.xlabel('PC-1') , plt.ylabel('PC-2')
plt.show()

## FID

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input

from skimage.transform import resize
from scipy.linalg import sqrtm

In [ ]:
#reference: https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/

def scale_images(images, new_shape):
	scaled_images = []
	for image in images:		
		new_image = resize(image, new_shape, 0)		
		scaled_images.append(new_image)
	return asarray(scaled_images)

def calculate_fid(model, images1, images2):
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

scaled_real = preprocess_input(scale_images(real_images, (299,299,3)))
scaled_coco = preprocess_input(scale_images(coco_images, (299,299,3)))
scaled_stylegan = preprocess_input(scale_images(stylegan_images, (299,299,3)))

fid_coco_real = calculate_fid(model, scaled_coco, scaled_real)
fid_stylegan_real = calculate_fid(model, scaled_stylegan, scaled_real)

In [ ]:
print("FID COCO:", fid_coco_real)
print("FID Stylegan:", fid_stylegan_real)

# Latent Images



In [ ]:
# Download
!curl https://storage.googleapis.com/dcgan_data/latent_images_coco.npy --output latent_images_coco.npy

In [ ]:
# Load
coco_latent = np.load("latent_images_coco.npy")

In [ ]:
coco_latent.shape

In [ ]:
def display_interpolated_images(images, count):
  w=256
  h=256
  fig=plt.figure(figsize=(5 * count, 7))

  for i in range(1, count +1):      
      img = images[i-1]
      fig.add_subplot(1, count, i)
      plt.axis('off')
      plt.imshow(img)  
        
  fig.subplots_adjust(wspace=0, hspace=0)
  plt.axis('off')    
  plt.show()

display_interpolated_images(coco_latent[49], 10)

###### Random tests



### Display means


In [ ]:
real_mean = real_images.mean(axis = 0)
coco_mean = coco_images.mean(axis = 0)
stylegan_mean = stylegan_images.mean(axis=0)

In [ ]:
display_image(real_mean)

In [ ]:
display_image(coco_mean)

In [ ]:
display_image(stylegan_mean)

In [ ]:
# Color gradient
gradient = np.ones((10, 256, 256, 3))

for i in range(10):
    gradient[i, :, :, 0] *= i/10;

gradient[:, :, :, 1] *= 0.6;
gradient[:, :, :, 2] *= 0.6;

display_interpolated_images(gradient, 10)